# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
# Upload city_data.csv from weatherpy
cities = pd.read_csv('city_data.csv')
cities.head(10)

,City SN,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hobart,75,AU,1584217206,81,-42.88,147.33,11.67,2.60
1,1,Araripe,100,BR,1584217295,93,-7.21,-40.05,22.97,1.71
2,2,Ongole,0,IN,1584217295,85,15.50,80.05,23.77,2.64
3,3,Nikolskoye,97,RU,1584217295,38,59.70,30.79,-3.00,10.00
4,4,Tuktoyaktuk,90,CA,1584217295,85,69.45,-133.04,-6.00,11.80
5,5,Kedrovyy,95,RU,1584217295,47,56.17,91.82,3.00,3.00
6,6,Omsukchan,100,RU,1584217295,97,62.53,155.80,-22.03,0.15
7,7,Yārāda,20,IN,1584217295,94,17.65,83.27,23.00,1.00
8,8,Bengkulu,95,ID,1584217296,91,-3.80,102.27,23.67,1.46
9,9,Hermanus,53,ZA,1584217296,88,-34.42,19.23,18.33,5.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Config gmaps with g_key
gmaps.configure(api_key=g_key)

# From cities df, store 'Latitude' and 'Longitude'

locations = cities[["Lat", "Lng"]]
humidity = cities["Humidity"].astype(float)


In [11]:
# Heat map, showing heat layers
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow down the cities to fit weather conditions.
cities_narrowed=cities.loc[(cities['Max Temp']>=0)&(cities['Max Temp']<=70) & (cities['Cloudiness']==0), :]
cities_narrowed = cities_narrowed.dropna(how='any')
cities_narrowed.reset_index(inplace=True)
del cities_narrowed['index']
cities_narrowed.head(5)

,City SN,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,2,Ongole,0,IN,1584217295,85,15.50,80.05,23.77,2.64
1,11,Nasīrābād,0,IN,1584217296,39,26.30,74.73,14.57,1.87
2,15,Birao,0,CF,1584217296,14,10.28,22.79,26.27,3.35
3,26,Gladstone,0,AU,1584217298,68,-23.85,151.25,21.00,2.60
4,35,Yulara,0,AU,1584217015,31,-25.24,130.99,25.00,5.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
# Creating a hotel file
hotels_in_cities = []

for i in range(len(cities_narrowed)):
    lat = cities_narrowed.loc[i]['Lat']
    lng = cities_narrowed.loc[i]['Lng']
    
# Used Parameters, lat & lng from cities_narrowed df
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    
    try:
       hotels_in_cities.append(jsn['results'][0]['name'])
    except:
        hotels_in_cities.append("")
cities_narrowed["Hotel Name"] = hotels_in_cities
cities_narrowed = cities_narrowed.dropna(how='any')
cities_narrowed.head()

,City SN,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,2,Ongole,0,IN,1584217295,85,15.50,80.05,23.77,2.64,Ongole
1,11,Nasīrābād,0,IN,1584217296,39,26.30,74.73,14.57,1.87,Nasirabad
2,15,Birao,0,CF,1584217296,14,10.28,22.79,26.27,3.35,Birao
3,26,Gladstone,0,AU,1584217298,68,-23.85,151.25,21.00,2.60,Gladstone
4,35,Yulara,0,AU,1584217015,31,-25.24,130.99,25.00,5.70,Petermann


In [41]:
# params = {
#     "location": (hotel_in_cities["Lat"], hotel_in_cities["Lng"]),
#     "rankby": "distance",
#     "type" : "hotel", 
#     "key" : g_key
# }

#  params
pprint(requested.json(), depth=1)

{'html_attributions': [],
 'next_page_token': 'CqQCGgEAAHKr2F4XC2l7Awi7bze7bT0z3-EOiiyNJ0voGNMlDgcmj7UtC5FpPqHs2s5yUcw21R80a5KNT5c5H7sZ_cKZPK76Dw9KxTmwTgEEy_jm2hyUKcPHnmIexrciXD-UQG59UoCQnmG19M4BqXA-zlKy_9M3MvYMp5UYq6bsnIu4jnFaOgzeh_ztx2mtnFJ25ZYC24Q_aSfhSAtr3IiQrS7GOuLOqfXTQNH75h_aWbqIFP-fPL-A0gSiK0fRRXv2E3NCUXzSDIE4YqRHi_oMB6kv4ywGvdlGlqXzeg-86y0ZLK5rNdxiYgGCqdofHHWi4ooJb0iCasFyb-A-VTh4Q3vqhdpBMQ76VZHZhTsThI6Ah8FR9Dcq0mHmXUqzZGAnPtCjjxIQO-49v_JDwq-8yHir2AtYNhoUIHNwYK8eQ8EKTywXoEIClQzcRiE',
 'results': [...],
 'status': 'OK'}


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels_in_cities = [info_box_template.format(**row) for index, row in cities_narrowed.iterrows()]
locations = cities_narrowed[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))